In [1]:
import pandas as pd
import openai
import requests
import time
from openai import OpenAI

In [2]:
client = OpenAI(
 api_key=<"add your api key here">, 
)


In [3]:
df=pd.read_excel("aps_data.xlsx")

In [4]:
df=df.fillna(" ")

In [5]:
address_cols=['street','city', 'zip', 'county']
df=df[address_cols]
df=df.astype(str)

In [6]:
df["zip"]=df["zip"].apply(lambda x:x.split("-")[0])

In [7]:
df=df[:50]

In [8]:
df["address"]=df["street"]+","+df["city"]+","+"TX "+df["zip"]+", USA"

In [11]:
def check_address(address):
    response=client.chat.completions.create(
    messages=[
            {
                "role": "user",
                "content": f"is this an address? if the answer is yes then correct this address and only return the corrected address or say no: {address}",
            }
        ],
        model="gpt-4o-mini",
    )
    return response.choices[0].message.content

def validate_address(address):
    result=None
    corrected_address=check_address(address)
    if corrected_address.lower()=="no.":
        return result
    print("after:",corrected_address)
    
    headers = {
        "User-Agent": "YourAppName/1.0 (your.email@example.com)"
    }

    url = f"https://nominatim.openstreetmap.org/search?q={corrected_address}&format=json"
    
    retries = 5
    backoff_factor = 1  # Start with a 1-second delay and increase on each retry
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 500:
            print(f"Internal Server Error. Retrying in {backoff_factor} seconds...")
            time.sleep(backoff_factor) 
            backoff_factor *= 2  
        elif response.status_code == 403:
            print("Access forbidden. Check your usage and User-Agent.")
            break
        elif response.status_code == 200:
            data = response.json()
            if data:
                print(f"Validated Address: {data[0]['display_name']}")
                result = data[0]['display_name']
                break
            else:
                time.sleep(backoff_factor)  
                backoff_factor *= 2  
        else:
            print(f"Error: Received status code {response.status_code}")
            break
    return result

In [12]:
df["validated_address"]=df["address"].apply(validate_address)

after: Denton, TX, USA
Validated Address: Denton, Denton County, Texas, United States
after: 8713 S Normandale St, Fort Worth, TX 76116, USA
Validated Address: 8713, South Normandale Street, Fort Worth, Tarrant County, Texas, 76116, United States
after: 8713 S Normandale St, Fort Worth, TX 76116, USA
Validated Address: 8713, South Normandale Street, Fort Worth, Tarrant County, Texas, 76116, United States
after: 8713 S Normandale St, Fort Worth, TX 76116, USA
Validated Address: 8713, South Normandale Street, Fort Worth, Tarrant County, Texas, 76116, United States
after: 401 W Sanford St, Arlington, TX 76011, USA
Validated Address: 401, West Sanford Street, Arlington, Tarrant County, Texas, 76011, United States
after: 1826 May St, Fort Worth, TX 76110, USA
Validated Address: 1826, May Street, Fort Worth, Tarrant County, Texas, 76110, United States
after: Denton, TX, USA
Validated Address: Denton, Denton County, Texas, United States
after: 1524 Morrison Dr, Fort Worth, TX 76112, USA
Valid